In [1]:
import torch
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import shutil
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torchvision
from PIL import Image
import glob
print(torch.__version__)
print(torch.cuda.is_available())
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

1.8.1+cu101
True


device(type='cuda', index=0)

- 卷积公式为：
$$w_{out} = \frac{w_{in} + 2p - f}{s} + 1$$
当s = 1, p = 0时：
$$w_{out} = w_{in} - （f - 1）$$
也即，此时卷积后的特征图相比较卷积前输入的特征图的大小来说，其大小要减少输入特征图减去(f-1)的大小，
- 因此，对于kernel_size = 3的卷积核来说，要想保持卷积前后特征图的尺寸不变，则$padding = \frac{(f-1)}{2} = 1$,
- 因此，对于kernel_size = 5的卷积核来说，要想保持卷积前后特征图的尺寸不变，则$padding = 2$

In [6]:
import torch.nn.functional as F
class ResNetBasicBlock(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super().__init__()
        # torch中padding=n代表对图像上下左右进行填充的像素数目
        self.conv1 = torch.nn.Conv2d(in_channels=in_channels, 
                                     out_channels=out_channels,
                                     kernel_size=3,
                                     padding=1,
                                     bias=False)
        self.batch_norm1 = torch.nn.BatchNorm2d(num_features=out_channels)
        self.conv2 = torch.nn.Conv2d(in_channels=out_channels, 
                                     out_channels=out_channels,
                                     kernel_size=3,
                                     padding=1,
                                     bias=False)
        self.batch_norm2 = torch.nn.BatchNorm2d(num_features=out_channels)
    
    def forward(self, input):
        residual = input
        x = self.conv1(input) 
        x = F.relu(self.batch_norm1(x), inplace=True) # inplace=True代表就地改变
        x = self.conv2(x) 
        x = self.batch_norm2(x)
        x += residual
        return F.relu(x)

In [12]:
model = ResNetBasicBlock(10, 20)